# Описание проекта

Нужно решить, где бурить новую скважину.

Предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Нужно проанализировать возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

# 1. Загрузка и подготовка данных

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy import stats as st

На данном это я добавляю необходимые библиотеки,который в дальнейшем потребуются для реализации проекта

In [3]:
data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_0 = data_0.drop_duplicates().reset_index(drop = True)
print(data_0.shape)
#data_0.head()

(100000, 5)


Здесь я делаю вычитку из 1 файла и смотрю на размеры таблицы и удаляю дубликаты

In [4]:
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_1 = data_1.drop_duplicates().reset_index(drop = True)
print(data_1.shape)
#data_1.head()

(100000, 5)


Здесь я делаю вычитку из 2 файла и смотрю на размеры таблицы и удаляю дубликаты

In [5]:
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
data_2 = data_2.drop_duplicates().reset_index(drop = True)
print(data_2.isna().sum())
#data_2.head()

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


Здесь я делаю вычитку из 3 файла и смотрю на размеры таблицы и удаляю дубликаты

In [5]:
df_train_0, df_valid_0 = train_test_split(data_0, test_size = 0.25, random_state=12345)

features_train_0 = df_train_0.drop(['product', 'id'], axis=1)
target_train_0 = df_train_0['product']
features_valid_0 = df_valid_0.drop(['product', 'id'], axis=1)
target_valid_0 = df_valid_0['product']

print(features_train_0.shape)
print(features_valid_0.shape)

(75000, 3)
(25000, 3)


На данном этапе я разбиваю на валидационную и тренеровочные выборки данных первого региона

In [6]:
df_train_1, df_valid_1 = train_test_split(data_1, test_size = 0.25, random_state=12345)

features_train_1 = df_train_1.drop(['product', 'id'], axis=1)
target_train_1 = df_train_1['product']
features_valid_1 = df_valid_1.drop(['product', 'id'], axis=1)
target_valid_1 = df_valid_1['product']

print(features_train_1.shape)
print(features_valid_1.shape)

(75000, 3)
(25000, 3)


На данном этапе я разбиваю на валидационную и тренеровочные выборки данных второго региона

In [7]:
df_train_2, df_valid_2 = train_test_split(data_2, test_size = 0.25, random_state=12345)

features_train_2 = df_train_2.drop(['product', 'id'], axis=1)
target_train_2 = df_train_2['product']
features_valid_2 = df_valid_2.drop(['product', 'id'], axis=1)
target_valid_2 = df_valid_2['product']

print(features_train_2.shape)
print(features_valid_2.shape)

(75000, 3)
(25000, 3)


На данном этапе я разбиваю на валидационную и тренеровочные выборки данных третьего региона

# 2. Обучение и проверка модели

In [8]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
valid_predicted_0 = model_0.predict(features_valid_0)
answer_0 = valid_predicted_0
answer_0 = pd.Series(answer_0)

rmse_0 = mean_squared_error(target_valid_0, valid_predicted_0) ** 0.5
print('RMSE = ', rmse_0)
print('Средний запас сырья: ', target_valid_0.mean())

RMSE =  37.5794217150813
Средний запас сырья:  92.07859674082927


In [9]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
valid_predicted_1 = model_1.predict(features_valid_1)

answer_1 = valid_predicted_1
answer_1 = pd.Series(answer_1)

rmse_1 = mean_squared_error(target_valid_1, valid_predicted_1) ** 0.5
print('RMSE = ', rmse_1)
print('Средний запас сырья: ', target_valid_1.mean())

RMSE =  0.893099286775616
Средний запас сырья:  68.72313602435997


In [10]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
valid_predicted_2 = model_2.predict(features_valid_2)

answer_2 = valid_predicted_2
answer_2 = pd.Series(answer_2)

rmse_2 = mean_squared_error(target_valid_2, valid_predicted_2) ** 0.5
print('RMSE = ', rmse_2)
print('Средний запас сырья: ', target_valid_2.mean())

RMSE =  40.02970873393434
Средний запас сырья:  94.88423280885438


На данном этапе я обучил модель линейной регрессии на тренеровочных выборках данных и вывожу на экран значение RMSE и среднего запаса сырья для каждого региона. Лучший результат получился у третьего региона

# 3. Подготовка к расчёту прибыли

In [11]:
min_product = 50000 / 450 
print('Минимальный обьем сырья для одного месторождения, чтобы оно было без убытка:', min_product)

Минимальный обьем сырья для одного месторождения, чтобы оно было без убытка: 111.11111111111111


In [12]:
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    profit_per_region = selected.sum() * 1000 * 450 - 10000000000
    return profit_per_region

Здесь я нашел минимальный обьем сырья для одного безубыточного месторождения и написал функцию для расчета прибыли

# 4. Расчёт прибыли и рисков 

In [13]:

state = np.random.RandomState(12345)
    
values_0 = []
for i in range(1000):
    target_subsample = target_valid_0.sample(n=500, replace=True, random_state=state)
    probs_subsample = answer_0[target_subsample.index]
    
    values_0.append(revenue(target_subsample, probs_subsample, 200))

values_0 = pd.Series(values_0)
lower_0 = values_0.quantile(0.01)

mean_0 = values_0.mean()

confidence_interval_0 = st.t.interval(0.95, len(values_0)-1, values_0.mean(), values_0.sem())

print("Средняя выручка:", mean_0)

print("95%-ый доверительный интервал:", confidence_interval_0)

print("1%-квантиль:", lower_0)

/opt/conda/lib/python3.7/site-packages/pandas/core/series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


Средняя выручка: -1727074054.4160826
95%-ый доверительный интервал: (-1745083898.5071185, -1709064210.3250468)
1%-квантиль: -2411688723.365426


In [14]:
values_1 = []
for i in range(1000):
    target_subsample = target_valid_1.sample(n=500, replace=True, random_state=state)
    probs_subsample = answer_1[target_subsample.index]
    
    values_1.append(revenue(target_subsample, probs_subsample, 200))

values_1 = pd.Series(values_1)
lower_1 = values_1.quantile(0.01)

mean_1 = values_1.mean()

confidence_interval_1 = st.t.interval(0.95, len(values_1)-1, values_1.mean(), values_1.sem())

print("Средняя выручка:", mean_1)

print("95%-ый доверительный интервал:", confidence_interval_1)

print("1%-квантиль:", lower_1)

Средняя выручка: -3811514804.935981
95%-ый доверительный интервал: (-3829893845.280592, -3793135764.5913696)
1%-квантиль: -4455315297.549019


In [15]:
values_2 = []
for i in range(1000):
    target_subsample = target_valid_2.sample(n=500, replace=True, random_state=state)
    probs_subsample = answer_2[target_subsample.index]
    
    values_2.append(revenue(target_subsample, probs_subsample, 200))

values_2 = pd.Series(values_2)
lower_2 = values_2.quantile(0.01)

mean_2 = values_2.mean()

confidence_interval_2 = st.t.interval(0.95, len(values_2)-1, values_2.mean(), values_2.sem())

print("Средняя выручка:", mean_2)

print("95%-ый доверительный интервал:", confidence_interval_2)

print("1%-квантиль:", lower_2)

Средняя выручка: -1480417773.5426087
95%-ый доверительный интервал: (-1498850856.8746321, -1461984690.2105854)
1%-квантиль: -2198850037.904337


На данном этапе я находил значения средней выручки, доверительного интервала и наиболее вероятный доход для каждого региона. Таким образом, на основании полученных показателей, можно сказать, что наиболее выгодный регион из трех является третий регион, поскольку значения показателей в данном регионе наиболее эффективные значения показателей по сравнению с другими и это говорит о том, что вероятнее всего наибольший доход компания получит в третьем регионе.